## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

In [4]:
df_lookup = spark.read.csv("/user/sprenner/lookup.csv", sep="\t")

In [5]:
#df_lookup.describe().toPandas()

In [6]:
#df_lookup.limit(10).toPandas()

In [7]:
df_lookup_renamed = df_lookup.toDF('SCOPE', 'NAME', 'CHILD_SCOPE', 'CHILD_NAME', 'DID_TYPE', 'CHILD_TYPE', 
                                   'LENGTH', 'BYTES', 'ADLER32', 'MD5', 'RULE_EVALUATION', 'UPDATED_AT', 
                                   'CREATED_AT', 'EVENTS')

In [8]:
df_april = spark.read.json("/user/sprenner/training_official_april.json")

In [9]:
df_april = df_april.withColumn("join_key_april", concat_ws(':', df_april["scope"], df_april["filename"]))
df_april = df_april.withColumn("fileScope", df_april["scope"])
df_april = df_april.drop("scope")
df_april.createOrReplaceTempView("april")
df_lookup_renamed = df_lookup_renamed.withColumn("join_key_lookup", concat_ws(':', df_lookup_renamed["CHILD_SCOPE"], df_lookup_renamed["CHILD_NAME"]))
df_lookup_renamed.createOrReplaceTempView("lookup_renamed")
df_lookup_renamed.count()

827642445

In [10]:
df_lookup_reduced = spark.sql("SELECT *\
                                 FROM lookup_renamed\
                                 WHERE SCOPE != 'panda' AND DID_TYPE == 'D' AND (NAME LIKE 'data%' OR NAME LIKE 'mc%')")
df_lookup_reduced.count()
# without official: 789897421

625826659

In [11]:
#df_joined_check_6 = df_joined_check_6.drop('LENGTH','ADLER32','MD5','RULE_EVALUATION','EVENTS')

In [12]:
df_lookup_reduced.createOrReplaceTempView("lookup_reduced")

dataset_count = spark.sql("SELECT *\
                      FROM (SELECT join_key_lookup AS join_key_lookup_count, COUNT(DISTINCT NAME) AS distinct_datasets\
                      FROM lookup_reduced \
                      GROUP BY join_key_lookup \
                      ORDER BY distinct_datasets DESC)")
dataset_count.limit(20).toPandas()
# oberstes: data10_7TeV:data10_7TeV.00152220.physics_L1CaloEM.merge.RAW._lb0002._0001.1     4

join_key_lookup_count  \
0   data10_7TeV:data10_7TeV.00152214.physics_L1Calo.merge.RAW._lb0171._0001.1                  
1   data10_cvalid:data10_7TeV.00152878.physics_RNDM.merge.RAW._lb0008._0001.1                  
2   data10_7TeV:data10_7TeV.00152508.physics_MinBias.merge.RAW._lb0191._0006.1                 
3   data10_cvalid:data10_7TeV.00152844.physics_MuonswBeam.merge.RAW._lb0167._0001.1            
4   data10_7TeV:data10_7TeV.00152220.physics_L1CaloEM.merge.RAW._lb0002._0001.1                
5   data10_cvalid:data10_7TeV.00152994.debug_EFHltTimeout.daq.RAW._lb0000._SFO-3._0001.data    
6   data17_13TeV:data17_13TeV.00339849.physics_EnhancedBias.merge.RAW._lb0173._SFO-4._0001.1   
7   data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0251._0005.1                  
8   data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0324._0004.1                  
9   data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0303._0007.1                  
10  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0289._0002.1                  
11  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0312._0003.1                  
12  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0258._0003.1                  
13  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0278._0002.1                  
14  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0259._0005.1                  
15  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0283._0007.1                  
16  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0256._0001.1                  
17  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0349._0007.1                  
18  data10_7TeV:data10_7TeV.00159224.physics_L1Calo.merge.RAW._lb0261._0005.1                  
19  data09_900GeV:data09_900GeV.00140571.physics_L1Calo.merge.RAW._lb0122._0001.1              

    distinct_datasets  
0   4                  
1   4                  
2   4                  
3   4                  
4   4                  
5   4                  
6   4                  
7   3                  
8   3                  
9   3                  
10  3                  
11  3                  
12  3                  
13  3                  
14  3                  
15  3                  
16  3                  
17  3                  
18  3                  
19  3

In [13]:
dataset_count.count()

622708625

In [14]:
dataset_count.createOrReplaceTempView("dataset_count")
df_lookup_dataset_count = spark.sql("SELECT * \
                                     FROM lookup_reduced JOIN dataset_count ON lookup_reduced.join_key_lookup == dataset_count.join_key_lookup_count")
df_lookup_dataset_count.limit(10).toPandas()

SCOPE  \
0  data08_1beammag   
1  data08_1beammag   
2  data08_1beammag   
3  data08_1beammag   
4  data08_1beammag   
5  data08_1beammag   
6  data08_1beammag   
7  data08_1beammag   
8  data08_1beammag   
9  data08_1beammag   

                                                                                          NAME  \
0  data08_1beammag.00087764.physics_MBTS_BCM_LUCID.recon.HIST.o4_r605_tid032361                  
1  data08_1beammag.00088128.physics_RNDM.recon.HIST.o4_r605_tid032373                            
2  data08_1beammag.00087764.physics_MBTS_BCM_LUCID.recon.HIST.o4_r653_tid047010                  
3  data08_1beammag.00087764.physics_L1Calo.merge.HIST.o4_r653_p18_tid053115                      
4  data08_1beammag.00087764.physics_MBTS_BCM_LUCID.recon.NTUP_MUONCALIB.o4_r653_r792_tid082443   
5  data08_1beammag.00088128.physics_MBTS_BCM_LUCID.recon.NTUP_MUONCALIB.o4_r653_r792_tid082454   
6  data08_1beammag.00088069.physics_BPTX.daq.RAW.o4                                              
7  data08_1beammag.00088069.physics_BPTX.daq.RAW.o4                                              
8  data08_1beammag.00088069.physics_MBTS_BCM_LUCID.daq.RAW.o4                                    
9  data08_1beammag.00088105.physics_RNDM.daq.RAW.o4                                              

       CHILD_SCOPE  \
0  data08_1beammag   
1  data08_1beammag   
2  data08_1beammag   
3  data08_1beammag   
4  data08_1beammag   
5  data08_1beammag   
6  data08_1beammag   
7  data08_1beammag   
8  data08_1beammag   
9  data08_1beammag   

                                                         CHILD_NAME DID_TYPE  \
0  HIST.032361._00106.root.1                                         D         
1  HIST.032373._00007.root.1                                         D         
2  HIST.047010._00130.root.2                                         D         
3  HIST.053115._00001.root.1                                         D         
4  NTUP_MUONCALIB.082443._000122.root.1                              D         
5  NTUP_MUONCALIB.082454._000032.root.1                              D         
6  daq.ATLAS.0088069.physics.BPTX.LB0024.SFO-4._0011.data            D         
7  daq.ATLAS.0088069.physics.BPTX.LB0024.SFO-5._0005.data            D         
8  daq.ATLAS.0088069.physics.MBTS_BCM_LUCID.LB0017.SFO-5._0001.data  D         
9  daq.ATLAS.0088105.physics.RNDM.LB0001.SFO-3._0001.data            D         

  CHILD_TYPE LENGTH      BYTES   ADLER32   MD5 RULE_EVALUATION     UPDATED_AT  \
0  F          0      32531211   6f8569e6  None  0               1230596278000   
1  F          0      20145988   8c777d93  None  0               1230478143000   
2  F          0      28173163   56b56cee  None  0               1238688027000   
3  F          0      40480899   b08ffd4b  None  0               1239062123000   
4  F          0      8678520    3bf797ba  None  0               1253776096000   
5  F          0      1843905    6341d337  None  0               1253774998000   
6  F          0      360377120  42bb71ac  None  0               1221176343000   
7  F          0      360452020  818a71a6  None  0               1221176343000   
8  F          0      1440936    a2ef591a  None  0               1221176354000   
9  F          0      233783784  23ebd90e  None  0               1221180269000   

      CREATED_AT EVENTS  \
0  1230596278000  None    
1  1230478143000  None    
2  1238688027000  None    
3  1239062123000  None    
4  1253776096000  None    
5  1253774998000  None    
6  1221176343000  1000    
7  1221176343000  1000    
8  1221176354000  4       
9  1221180269000  16      

                                                                    join_key_lookup  \
0  data08_1beammag:HIST.032361._00106.root.1                                          
1  data08_1beammag:HIST.032373._00007.root.1                                          
2  data08_1beammag:HIST.047010._00130.root.2                                          
3  data08_1beammag:HIST.05311

In [16]:
df_lookup_dataset_count.count()

625826659

In [17]:
df_april.count()

777286

In [18]:
df_lookup_dataset_count.createOrReplaceTempView("count")
df_lookup_final = spark.sql("SELECT *\
                             FROM april LEFT JOIN count ON april.join_key_april == count.join_key_lookup_count")# AND count.distinct_datasets == 1")
df_lookup_final.count()

780484

In [ ]:
df_lookup_final.createOrReplaceTempView("final")
spark.sql("SELECT * FROM final WHERE distinct_datasets IS NULL").limit(20).toPandas()

In [ ]:
spark.sql("SELECT * FROM final WHERE distinct_datasets > 1").toPandas()

In [ ]:
df_lookup_final.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_lookup_final.columns]).limit(10).toPandas()

## check for multiple records

In [ ]:
spark.sql("SELECT join_key_april, COUNT(*)\
           FROM final \
           GROUP BY join_key_april\
           ORDER BY 2 DESC").limit(20).toPandas()

In [ ]:
temptest = spark.sql("SELECT join_key_april, COUNT(*)\
           FROM final \
           GROUP BY join_key_april\
           ORDER BY 2 DESC")
temptest.where("join_key_april == 'mc16_13TeV:NTUP_PILEUP.13658010._000008.pool.root.1'").toPandas()

In [ ]:
spark.sql("SELECT *\
           FROM final \
           WHERE join_key_april == 'mc16_5TeV:DAOD_EGAM1.13695903._000004.pool.root.1'").limit(20).toPandas()

In [ ]:
df_lookup_final.describe().toPandas()

In [ ]:
#df_lookup_final.write.format("json").save("/user/sprenner/lookup_table_1_official.json")